### Real Case Study



### Instance generation

In [ ]:
GRID_SIZE = 9
FILENAME = 'realCase'
STATIC_WITNESSES = 25
TESTIMONIES_PER_LYING_WITNESS = 5
BASE_SIMILARITIES = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
LYING_WITNESSES = [0,1,2,3,4,5]
NUMBER_OF_INSTANCES = len(BASE_SIMILARITIES) * len(LYING_WITNESSES)

In [ ]:
from src.realisticCase import RealisticCaseInstanceGenerator, RealisticCaseSolutionData
from src.witnessproblem import Instance
import random

random.seed(1234)

instances = []
routes: list[RealisticCaseSolutionData] = []
for base_similarity in BASE_SIMILARITIES:
    for lying_witnesses in LYING_WITNESSES:
        instance_generator = RealisticCaseInstanceGenerator(
            base_similarity=base_similarity, 
            grid_size=GRID_SIZE, 
            lying_witnesses=lying_witnesses,
            static_witnesses=STATIC_WITNESSES,
            testimonies_per_lying_witness=TESTIMONIES_PER_LYING_WITNESS)
        instance, actor_route = instance_generator.generateInstance()

        instances.append(instance)
        routes.append(RealisticCaseSolutionData(base_similarity,lying_witnesses,actor_route))

with open(f"instances/{FILENAME}.txt",'w') as instance_file, open(f"solutions/{FILENAME}.txt",'w') as solution_file:
    instance_file.write(Instance.schema().dumps(instances, many=True))
    solution_file.write(RealisticCaseSolutionData.schema().dumps(routes, many=True))